In [4]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3", base_url="http://localhost:11434")
print(llm.invoke("Say hello"))


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError("HTTPConnection(host='localhost', port=11434): Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it"))

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import uuid

def hierarchical_chunking(docs):
    parent_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""]
    )

    child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=350,
        chunk_overlap=80,
        separators=["\n\n", "\n", ". ", " ", ""]
    )

    chunks = []

    for doc in docs:
        parents = parent_splitter.split_text(doc.page_content)

        for parent in parents:
            parent_id = str(uuid.uuid4())
            children = child_splitter.split_text(parent)

            for child in children:
                chunks.append(
                    Document(
                        page_content=child,
                        metadata={
                            "parent_id": parent_id,
                            "source": doc.metadata.get("source"),
                            "page": doc.metadata.get("page"),
                        }
                    )
                )
    return chunks


# Load PDF
loader = PyPDFLoader("HR_Policies_2025.pdf")
docs = loader.load()

chunks = hierarchical_chunking(docs)
print(f"Total chunks: {len(chunks)}")
